In [ ]:
import numpy as np
import geopandas as gpd
import rasterio
from pygeohydro import WBD

In [ ]:
# lat, lon = 47.5, -121.8   # example WA Cascades

lat, lon = 48.310760, -120.657464

wbd = WBD("huc12")
huc12 = wbd.bygeom((lon, lat), geo_crs=4326)  # point query
huc12


In [ ]:
huc12.plot()

In [ ]:
list(huc12)

In [ ]:
from pygeohydro import WBD

lat, lon = 48.310760, -120.657464

wbd8 = WBD("huc8")
huc8 = wbd8.bygeom((lon, lat), geo_crs=4326)
huc8

In [ ]:
huc8.plot()

In [ ]:
import folium

# If huc8 is a GeoDataFrame (it is), reproject to WGS84 for folium
huc8_wgs = huc12.to_crs(4326)

m = folium.Map(location=[lat, lon], zoom_start=8, tiles="CartoDB positron")

# add polygon
folium.GeoJson(
    huc8_wgs,
    name="HUC8",
    style_function=lambda x: {"weight": 2, "fillOpacity": 0.15},
).add_to(m)

# add query point
folium.Marker([lat, lon], tooltip="Query point").add_to(m)

folium.LayerControl().add_to(m)
m

In [ ]:
# import geopandas as gpd
# from pygeohydro import WBD

# # 1) Get Washington state polygon (WGS84)
# wa = gpd.read_file("https://github.com/PublicaMundi/MappingAPI/raw/master/data/us-states.json")
# wa = wa[wa["name"] == "Washington"].to_crs(4326)

# # 2) Query WBD for all HUC12s intersecting WA
# wbd12 = WBD("huc12")
# wa_huc12 = wbd12.bygeom(wa.geometry.iloc[0], geo_crs=4326)

# wa_huc12

In [ ]:
# import geopandas as gpd
# import pandas as pd
# from pygeohydro import WBD

# # 1) Washington polygon (WGS84)
# states_url = "https://www2.census.gov/geo/tiger/GENZ2023/shp/cb_2023_us_state_20m.zip"
# states = gpd.read_file(states_url).to_crs(4326)

# wa = states[states["STUSPS"] == "WA"].copy()
# wa_geom = wa.geometry.iloc[0]  # can be MultiPolygon

# # 2) Split WA into Polygon parts (because bygeom() may not accept MultiPolygon)
# if wa_geom.geom_type == "MultiPolygon":
#     wa_parts = list(wa_geom.geoms)
# else:
#     wa_parts = [wa_geom]

# # 3) Query HUC12 intersecting WA (union of all parts)
# wbd12 = WBD("huc12")
# pieces = []
# for poly in wa_parts:
#     pieces.append(wbd12.bygeom(poly, geo_crs=4326))

# huc12_wa = gpd.GeoDataFrame(pd.concat(pieces, ignore_index=True), crs=4326)

# # de-duplicate by HUC id (sometimes overlaps between parts)
# if "huc12" in huc12_wa.columns:
#     huc12_wa = huc12_wa.drop_duplicates(subset=["huc12"])

# # 4) Clip to WA boundary (final clean WA-only geometry)
# huc12_wa_clipped = gpd.clip(huc12_wa, wa.to_crs(4326))

# print("HUC12 rows (intersect WA):", len(huc12_wa))
# print("HUC12 rows (clipped):", len(huc12_wa_clipped))

# huc12_wa_clipped.head()

In [ ]:
# --- Get HUC12 polygons within a bounding box (no WA clipping) ---

minx, miny, maxx, maxy = -123.6, 46.6, -119.9, 49.4
bbox = box(minx, miny, maxx, maxy)

# 2) Query WBD for HUC12 features intersecting the bbox
wbd12 = WBD("huc12")
huc12 = wbd12.bygeom(bbox, geo_crs=4326)  # returns a GeoDataFrame

print("HUC12 rows:", len(huc12))
huc12.head()

# 3) (Optional) hard-clip geometries to the bbox
bbox_gdf = gpd.GeoDataFrame(geometry=[bbox], crs=4326)
huc12_clip = gpd.clip(huc12.to_crs(4326), bbox_gdf)

print("Clipped rows:", len(huc12_clip))
huc12_clip.head()

In [ ]:
from pathlib import Path

import geopandas as gpd
import rasterio
from rasterio.mask import mask
from bmi_topography import Topography


def fetch_clipped_dem_for_aoi(
    aoi_path: str,
    out_dir: str = r"/mnt/c/Users/amehedi/Downloads",
    dem_type: str = "SRTMGL1",
    output_format: str = "GTiff",
    buffer_deg: float = 0.0,
    api_key: str | None = None,
    cache_dir: str | None = None,
) -> str:
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    cache_dir = Path(cache_dir) if cache_dir else out_dir / "dem_cache"
    cache_dir.mkdir(parents=True, exist_ok=True)

    gdf = gpd.read_file(aoi_path)
    gdf_wgs84 = gdf.to_crs(epsg=4326)
    west, south, east, north = gdf_wgs84.total_bounds

    topo = Topography(
        dem_type=dem_type,
        south=south - buffer_deg,
        north=north + buffer_deg,
        west=west - buffer_deg,
        east=east + buffer_deg,
        output_format=output_format,
        cache_dir=str(cache_dir),
        api_key=api_key,
    )
    dem_path = Path(topo.fetch())

    with rasterio.open(dem_path) as src:
        gdf_src = gdf.to_crs(src.crs)
        shapes = [geom.__geo_interface__ for geom in gdf_src.geometry]
        out_image, out_transform = mask(src, shapes, crop=True)

        out_meta = src.meta.copy()
        out_meta.update(
            {
                "driver": "GTiff",
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform,
            }
        )

    out_path = out_dir / f"dem_{dem_type.lower()}_clipped.tif"
    with rasterio.open(out_path, "w", **out_meta) as dst:
        dst.write(out_image)

    return str(out_path)

In [ ]:
dem_path = fetch_clipped_dem_for_aoi(
    r"/mnt/c/Users/amehedi/Downloads/purple_creek_da.gpkg"
)
print(dem_path)


In [ ]:
from pathlib import Path

import geopandas as gpd
import numpy as np
import rasterio
from rasterio.mask import mask
from bmi_topography import Topography


def _write_esri_ascii(
    array_2d: np.ndarray,
    transform,
    out_path: Path,
    nodata_value: float,
) -> str:
    height, width = array_2d.shape
    xllcorner = transform.c
    yllcorner = transform.f + transform.e * height
    cellsize = abs(transform.a)

    with open(out_path, "w") as f:
        f.write(f"ncols         {width}\n")
        f.write(f"nrows         {height}\n")
        f.write(f"xllcorner     {xllcorner}\n")
        f.write(f"yllcorner     {yllcorner}\n")
        f.write(f"cellsize      {cellsize}\n")
        f.write(f"NODATA_value  {nodata_value}\n")
        for row in array_2d:
            f.write(" ".join(map(str, row)) + "\n")

    return str(out_path)


def _force_overwrite(path: Path) -> None:
    if path.exists():
        try:
            path.unlink()
        except Exception as e:
            raise PermissionError(
                f"Could not overwrite {path}. Close the file (QGIS/ArcGIS/preview) and retry."
            ) from e


def fetch_clipped_dem_for_aoi(
    aoi_path: str,
    out_dir: str = r"/mnt/c/Users/amehedi/Downloads",
    dem_type: str = "SRTMGL1",
    output_format: str = "GTiff",
    buffer_deg: float = 0.0,
    api_key: str | None = None,
    cache_dir: str | None = None,
    nodata_value: float = -9999.0,
) -> tuple[str, str]:
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    cache_dir = Path(cache_dir) if cache_dir else out_dir / "dem_cache"
    cache_dir.mkdir(parents=True, exist_ok=True)

    gdf = gpd.read_file(aoi_path)
    gdf_wgs84 = gdf.to_crs(epsg=4326)
    west, south, east, north = gdf_wgs84.total_bounds

    topo = Topography(
        dem_type=dem_type,
        south=south - buffer_deg,
        north=north + buffer_deg,
        west=west - buffer_deg,
        east=east + buffer_deg,
        output_format=output_format,
        cache_dir=str(cache_dir),
        api_key=api_key,
    )
    dem_path = Path(topo.fetch())

    with rasterio.open(dem_path) as src:
        gdf_src = gdf.to_crs(src.crs)
        shapes = [geom.__geo_interface__ for geom in gdf_src.geometry]
        out_image, out_transform = mask(src, shapes, crop=True)

        nd = src.nodata if src.nodata is not None else nodata_value

        out_meta = src.meta.copy()
        out_meta.update(
            {
                "driver": "GTiff",
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform,
                "nodata": nd,
            }
        )

    out_tif = out_dir / f"dem_{dem_type.lower()}_clipped.tif"
    out_asc = out_dir / f"dem_{dem_type.lower()}_clipped.asc"

    _force_overwrite(out_tif)
    _force_overwrite(out_asc)

    with rasterio.open(out_tif, "w", **out_meta) as dst:
        dst.write(out_image)

    arr = out_image[0].astype(float)
    arr[arr == nd] = np.nan
    arr_out = np.where(np.isnan(arr), nd, arr)

    _write_esri_ascii(arr_out, out_transform, out_asc, nd)

    return str(out_tif), str(out_asc)


# ---- example ----
tif_path, asc_path = fetch_clipped_dem_for_aoi(
    r"/mnt/c/Users/amehedi/Downloads/purple_creek_da.gpkg"
)
print(tif_path)
print(asc_path)

In [ ]:
import numpy as np
import rasterio
import matplotlib.pyplot as plt

tif_path = r"/mnt/c/Users/amehedi/Downloads/dem_srtmgl1_clipped.tif"
asc_path = r"/mnt/c/Users/amehedi/Downloads/dem_srtmgl1_clipped.asc"

def quick_plot_tif(path):
    with rasterio.open(path) as src:
        z = src.read(1).astype(float)
        if src.nodata is not None:
            z[z == src.nodata] = np.nan
    print("TIF min/max:", np.nanmin(z), np.nanmax(z))
    plt.figure(figsize=(6,5))
    plt.imshow(z, cmap="terrain")
    plt.colorbar(label="Elevation")
    plt.title("Clipped DEM (TIF)")
    plt.axis("off")
    plt.show()

def quick_plot_asc(path):
    with open(path) as f:
        header = [next(f) for _ in range(6)]
    nodata = float([h for h in header if "NODATA" in h.upper()][0].split()[-1])
    z = np.loadtxt(path, skiprows=6)
    z[z == nodata] = np.nan
    print("ASC min/max:", np.nanmin(z), np.nanmax(z))
    plt.figure(figsize=(6,5))
    plt.imshow(z, cmap="terrain")
    plt.colorbar(label="Elevation")
    plt.title("Clipped DEM (ASC)")
    plt.axis("off")
    plt.show()

quick_plot_tif(tif_path)
quick_plot_asc(asc_path)

In [ ]:
from pathlib import Path

import geopandas as gpd
import numpy as np
import rasterio
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
from bmi_topography import Topography


def _write_esri_ascii(array_2d, transform, out_path, nodata_value):
    height, width = array_2d.shape
    xllcorner = transform.c
    yllcorner = transform.f + transform.e * height
    cellsize = abs(transform.a)

    with open(out_path, "w") as f:
        f.write(f"ncols         {width}\n")
        f.write(f"nrows         {height}\n")
        f.write(f"xllcorner     {xllcorner}\n")
        f.write(f"yllcorner     {yllcorner}\n")
        f.write(f"cellsize      {cellsize}\n")
        f.write(f"NODATA_value  {nodata_value}\n")
        for row in array_2d:
            f.write(" ".join(map(str, row)) + "\n")

    return str(out_path)


def _force_overwrite(path: Path):
    if path.exists():
        path.unlink()


def fetch_clipped_dem_for_aoi(
    aoi_path: str,
    out_dir: str,
    dem_type: str = "USGS10m",
    target_resolution: float = 10.0,
    buffer_deg: float = 0.05,
    api_key: str | None = None,
    nodata_value: float = -9999.0,
):
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    # ---- AOI + UTM detection ----
    gdf = gpd.read_file(aoi_path)

    centroid = gdf.to_crs(4326).geometry.unary_union.centroid
    zone = int((centroid.x + 180) / 6) + 1
    epsg = 32600 + zone if centroid.y >= 0 else 32700 + zone
    gdf_utm = gdf.to_crs(epsg=epsg)

    # ---- Download DEM ----
    west, south, east, north = gdf.to_crs(4326).total_bounds

    topo = Topography(
        dem_type=dem_type,
        south=south - buffer_deg,
        north=north + buffer_deg,
        west=west - buffer_deg,
        east=east + buffer_deg,
        output_format="GTiff",
        api_key=api_key,
    )
    dem_path = Path(topo.fetch())

    # ---- Clip DEM ----
    with rasterio.open(dem_path) as src:
        shapes = [geom.__geo_interface__ for geom in gdf.to_crs(src.crs).geometry]
        clipped, clip_transform = mask(src, shapes, crop=True)
        src_crs = src.crs
        src_nodata = src.nodata if src.nodata is not None else nodata_value

    # ---- Reproject + resample to UTM ----
    dst_transform, width, height = calculate_default_transform(
        src_crs,
        f"EPSG:{epsg}",
        clipped.shape[2],
        clipped.shape[1],
        *rasterio.transform.array_bounds(
            clipped.shape[1], clipped.shape[2], clip_transform
        ),
        resolution=target_resolution,
    )

    dst = np.empty((1, height, width), dtype=np.float32)

    reproject(
        source=clipped,
        destination=dst,
        src_transform=clip_transform,
        src_crs=src_crs,
        dst_transform=dst_transform,
        dst_crs=f"EPSG:{epsg}",
        resampling=Resampling.bilinear,
        src_nodata=src_nodata,
        dst_nodata=src_nodata,
    )

    # ---- Write outputs ----
    out_tif = out_dir / f"dem_utm_{int(target_resolution)}m.tif"
    out_asc = out_dir / f"dem_utm_{int(target_resolution)}m.asc"

    _force_overwrite(out_tif)
    _force_overwrite(out_asc)

    with rasterio.open(
        out_tif,
        "w",
        driver="GTiff",
        height=height,
        width=width,
        count=1,
        dtype=dst.dtype,
        crs=f"EPSG:{epsg}",
        transform=dst_transform,
        nodata=src_nodata,
    ) as ds:
        ds.write(dst)

    arr = dst[0]
    arr[np.isnan(arr)] = src_nodata
    _write_esri_ascii(arr, dst_transform, out_asc, src_nodata)

    return str(out_tif), str(out_asc)


In [ ]:
tif, asc = fetch_clipped_dem_for_aoi(
    aoi_path="/mnt/c/Users/amehedi/Downloads/purple_creek_da.gpkg",
    out_dir="/mnt/c/Users/amehedi/Downloads",
    api_key="985ca66e17da97a20d0732c993597ea3"
)

print(tif)
print(asc)

In [ ]:
import rasterio
import matplotlib.pyplot as plt
import numpy as np


def inspect_dem(dem_path):
    """
    Print CRS, resolution, bounds, shape
    """
    with rasterio.open(dem_path) as src:
        print("CRS:", src.crs)
        print("EPSG:", src.crs.to_epsg())
        print("Resolution (x, y):", src.res)
        print("Width, Height:", src.width, src.height)
        print("Bounds:", src.bounds)
        print("NoData:", src.nodata)


def plot_dem(dem_path, cmap="terrain"):
    """
    Quick DEM map plot
    """
    with rasterio.open(dem_path) as src:
        dem = src.read(1)
        dem = np.ma.masked_equal(dem, src.nodata)

    plt.figure(figsize=(8, 6))
    plt.imshow(dem, cmap=cmap)
    plt.colorbar(label="Elevation (m)")
    plt.title("DEM")
    plt.axis("off")
    plt.tight_layout()
    plt.show()


# ---- usage ----
dem_path = "/mnt/c/Users/amehedi/Downloads/dem_utm_10m.tif"

inspect_dem(dem_path)
plot_dem(dem_path)

In [ ]:
import rasterio
import matplotlib.pyplot as plt
import numpy as np


def plot_dem_simple(dem_path):
    with rasterio.open(dem_path) as src:
        dem = src.read(1)
        dem = np.ma.masked_equal(dem, src.nodata)
        extent = [
            src.bounds.left,
            src.bounds.right,
            src.bounds.bottom,
            src.bounds.top,
        ]

    plt.figure(figsize=(6, 6))
    plt.imshow(dem, extent=extent, origin="upper", cmap="terrain")
    plt.colorbar(label="Elevation (m)")
    plt.xlabel("X (m)")
    plt.ylabel("Y (m)")
    plt.title("Elevation")
    plt.tight_layout()
    plt.show()


# ---- usage ----
plot_dem_simple("/mnt/c/Users/amehedi/Downloads/dem_utm_10m.tif")


In [ ]:
import pandas as pd

csv_path = "/mnt/c/Users/amehedi/Downloads/Change_Detection_Data_Release.csv"
df = pd.read_csv(csv_path)
df.columns

subset = df[df["Data_Availability_Category"] == "pre_and_post"][["Fire_ID", "Fire_Name"]]
subset